# Project 2: Recommender System

## I. Data Pre-processing

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3 as sql
import difflib
import re
import json
import os
from underthesea import word_tokenize, pos_tag, sent_tokenize
import regex
import demoji
from pyvi import ViPosTagger, ViTokenizer
import string
import import_ipynb
import lib_nlp as func
import warnings
warnings.filterwarnings('ignore')

importing Jupyter notebook from lib_nlp.ipynb


## I.1. Product

In [2]:
products = pd.read_csv('data/ProductRaw.csv')

In [3]:
products.head()

,item_id,name,description,rating,price,list_price,brand,group,url,image
0,48102821,Tai nghe Bluetooth Inpods 12 - Cảm biến vân ta...,THÔNG TIN CHI TIẾT\nDung lượng pin 300\nThời g...,4.0,77000,300000,OEM,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,https://tai-nghe-bluetooth-inpods-12-cam-bien-...,https://salt.tikicdn.com/cache/280x280/ts/prod...
1,52333193,Tai nghe bluetooth không dây F9 True wireless ...,THÔNG TIN CHI TIẾT\nDung lượng pin 2000mah\nTh...,4.5,132000,750000,OEM,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,https://tai-nghe-bluetooth-khong-day-f9-true-w...,https://salt.tikicdn.com/cache/280x280/ts/prod...
2,299461,Chuột Không Dây Logitech M331 Silent - Hàng Ch...,THÔNG TIN CHI TIẾT\nThương hiệu Logitech\nĐộ p...,4.8,299000,399000,Logitech,Thiết Bị Số - Phụ Kiện Số/Phụ kiện máy tính và...,https://chuot-khong-day-logitech-m331-silent-p...,https://salt.tikicdn.com/cache/280x280/media/c...
3,57440329,Loa Bluetooth 5.0 Kiêm Đồng Hồ Báo Thức - [[ 2...,THÔNG TIN CHI TIẾT\nThương hiệu Acome\nXuất xứ...,4.7,149000,350000,Acome,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,https://loa-bluetooth-5-0-kiem-dong-ho-bao-thu...,https://salt.tikicdn.com/cache/280x280/ts/prod...
4,38458616,Tai Nghe Bluetooth Apple AirPods Pro True Wire...,THÔNG TIN CHI TIẾT\nThương hiệu Apple\nXuất xứ...,4.8,5090000,8500000,Apple,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,https://tai-nghe-bluetooth-apple-airpods-pro-t...,https://salt.tikicdn.com/cache/280x280/ts/prod...


In [4]:
print('Number of products: ', products.shape[0])

Number of products:  4404


In [5]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4404 entries, 0 to 4403
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   item_id      4404 non-null   int64  
 1   name         4404 non-null   object 
 2   description  4401 non-null   object 
 3   rating       4404 non-null   float64
 4   price        4404 non-null   int64  
 5   list_price   4404 non-null   int64  
 6   brand        4404 non-null   object 
 7   group        4404 non-null   object 
 8   url          4404 non-null   object 
 9   image        4404 non-null   object 
dtypes: float64(1), int64(3), object(6)
memory usage: 344.2+ KB


In [6]:
products.describe(include='O')

,name,description,brand,group,url,image
count,4404,4401,4404,4404,4404,4404
unique,4330,4370,521,248,4373,4362
top,Chuột Không Dây Logitech M171 - Hàng Chính Hãng,THÔNG TIN CHI TIẾT\nThương hiệu Logitech\nXuất...,OEM,Máy Ảnh - Máy Quay Phim/Camera Giám Sát/Camera IP,https://ban-phim-g20-led-gia-co-game-chuyen-du...,https://salt.tikicdn.com/cache/280x280/ts/prod...
freq,3,2,1121,236,2,2


**Checking for duplicates if any present**

In [7]:
print('Shape before dropping duplicates: ', products.shape)
products = products.drop_duplicates()
print('Shape after dropping duplicates: ', products.shape)

Shape before dropping duplicates:  (4404, 10)
Shape after dropping duplicates:  (4373, 10)


**Checking nulls**

In [8]:
print(np.sum(products.isnull()), '\n')

item_id        0
name           0
description    3
rating         0
price          0
list_price     0
brand          0
group          0
url            0
image          0
dtype: int64 



Chỉ có 3 sản phẩm bị thiếu 'description' => Không đáng kể

**Saving the result**

In [9]:
products.to_csv('data/ProductNew.csv', index=False)

## I.2. Review

In [10]:
reviews = pd.read_csv('data/ReviewRaw.zip')

In [11]:
reviews.head()

,customer_id,product_id,name,full_name,created_time,rating,title,content
0,709310,10001012,Lân Nguyễn Hoàng,Lân Nguyễn Hoàng,NaN,3,Ko dùng đc thẻ nhớ,Lúcđầu quên thông tin nên dùng 512gb thì ko đc...
1,10701688,10001012,Nguyễn Khánh Hòa,Nguyễn Khánh Hòa,NaN,5,Cực kì hài lòng,"Tiki giao hàng nhanh. Sản phẩm đúng như mô tả,..."
2,11763074,10001012,Toàn Phạm Khánh,Toàn Phạm Khánh,2019-04-17 15:42:45,5,Cực kì hài lòng,"chất lượng camera rõ nét, chống mưa nắng tuyệt..."
3,9909549,10001012,Nguyen Quang Minh,NaN,NaN,5,Rất hài lòng,"Hàng được đóng gói cẩn thận, giao hàng nhanh ,..."
4,1827148,10001012,Phạm Bá Đức,Phạm Bá Đức,NaN,5,Cực kì hài lòng,"dễ cài đặt, chất lượng tốt, chế độ xem hồng ng..."


In [12]:
print('Number of reviews: ', reviews.shape[0])

Number of reviews:  364099


In [13]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 364099 entries, 0 to 364098
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   customer_id   364099 non-null  int64 
 1   product_id    364099 non-null  int64 
 2   name          363669 non-null  object
 3   full_name     329471 non-null  object
 4   created_time  117340 non-null  object
 5   rating        364099 non-null  int64 
 6   title         364070 non-null  object
 7   content       165794 non-null  object
dtypes: int64(3), object(5)
memory usage: 22.2+ MB


In [14]:
reviews.describe(include='O')

,name,full_name,created_time,title,content
count,363669,329471,117340,364070,165794
unique,165632,153708,92636,27480,150821
top,Tài khoản Zalo,Tài khoản Zalo,2018-06-29 11:50:44,Cực kì hài lòng,ok
freq,830,830,29,229463,1531


**Checking for duplicates if any present**

In [15]:
print('Shape before dropping duplicates: ', reviews.shape)
reviews = reviews.drop_duplicates()
print('Shape after dropping duplicates: ', reviews.shape)

Shape before dropping duplicates:  (364099, 8)
Shape after dropping duplicates:  (361750, 8)


**Checking nulls**

In [16]:
print(np.sum(reviews.isnull()), '\n')

customer_id          0
product_id           0
name               428
full_name        34603
created_time    245525
rating               0
title               29
content         196064
dtype: int64 



Có nhiều đánh giá bị thiếu thông tin ở cột 'name', 'full_name', 'created_time', 'title' và 'content'
- 'created_time' có dữ liệu null khá lớn
- 'full_name' có dữ liệu null tương đối

=> Cần xử lý

In [17]:
x = []
for y in['full_name', 'created_time']:
    x.append({
        'feature': y,
        'missing_value': reviews[reviews[y].isnull()].shape[0],
        'percentage': reviews[reviews[y].isnull()].shape[0]/reviews.shape[0]*100
    })

df = pd.DataFrame(x)
df

,feature,missing_value,percentage
0,full_name,34603,9.565446
1,created_time,245525,67.871458


In [18]:
# Removing feature full_name and created_time
reviews.drop(['full_name', 'created_time'], axis=1, inplace=True)

**Processing review_text**

In [20]:
reviews['content_clean'] = reviews.content.apply(lambda x: func.clean_text_thesea(str(x), func.emoji_dict, func.teen_dict, func.wrong_lst, func.stopwords_lst))

reviews.head(3)

,customer_id,product_id,name,rating,title,content,content_clean
0,709310,10001012,Lân Nguyễn Hoàng,3,Ko dùng đc thẻ nhớ,Lúcđầu quên thông tin nên dùng 512gb thì ko đc...,lúcđầu quên thông không_đc đọc khởi tóm khởi
1,10701688,10001012,Nguyễn Khánh Hòa,5,Cực kì hài lòng,"Tiki giao hàng nhanh. Sản phẩm đúng như mô tả,...",tốt bắt hình_ảnh rộng hài_lòng tưởng
2,11763074,10001012,Toàn Phạm Khánh,5,Cực kì hài lòng,"chất lượng camera rõ nét, chống mưa nắng tuyệt...",chất_lượng chống disconect


In [21]:
reviews = reviews[reviews.content_clean != ' ']
reviews = reviews[reviews.content_clean != '']

**Saving the result**

In [22]:
reviews.to_csv('data/ReviewNew.zip', index=False)

## I.3. Product - Review

In [23]:
products = pd.read_csv('data/ProductNew.csv')
reviews = pd.read_csv('data/ReviewNew.zip')

**The products are not in the reviews**

In [24]:
reviews[~reviews.product_id.isin(products.item_id)]

,customer_id,product_id,name,rating,title,content,content_clean
63734,5687133,20845056,Lê Phương Tiến,4,Hài lòng,Hài lòng về máy sau 1 tuần dùng. Điểm trừ giao...,hài_lòng trừ quên thông khuyên sạc sài đồng_bộ...
63735,8344149,20845056,Luu Tri Dung,5,Tuyệt vời,"Giao hàng đúng thời gian dự kiến, có 2 ngày. M...",đọc nhập không_khác phân_phối chính_thức rẻ tố...
63736,10671105,20845056,Nguyễn Thu Hằng,5,Cực kì hài lòng,"Sản phẩm được đóng gói cẩn thận, pin dùng được...",đóng_gói giơ không_loá tích_hợp học chống gọn ...
63748,373425,20845056,Quân Nguyễn,2,Nên mua ở ngoài,Mình mua kindle Paperwhite 2019 được tầm cỡ 2 ...,kindle tầm_cỡ gửi sạc đọc kindle sạc không_đọc...
63756,1279789,20845056,Quân Trần,1,"Ship chậm, máy ko active được",Tôi nhận đc sản phẩm ko active được. Khi liên ...,không_active liên_hệ báo đánh_cắp khoá không_h...
...,...,...,...,...,...,...,...
78731,13809300,12706787,Nguyễn Thanh Liêm,3,Bình thường,"chắc đeo hơi đau tai hả shop, vì tai nghe kg c...",đeo hơi đau hả_shop mềm bọc
78732,8292666,12706787,huy tran quang,3,Bình thường,Đây là hàng chính hãng việt nam hay hàng quốc ...,quốc_tế
78733,6079274,12706787,Nguyễn trung hiếu,4,"Hàng nghe mượt, tốt",Hàng chuẩn! Nhưng mình đang phân vân ko biết 1...,chuẩn phân_vân
78734,7474878,12706787,Mon Pham,5,Quá phê !!!!,Mong lần sau đừng giao hàng chậm nữa tháng như...,đừng chậm


**Dropping the reviews having unknown item_id**

In [25]:
reviews = reviews[reviews.product_id.isin(products.item_id)]

In [26]:
reviews.reset_index(drop=True, inplace=True)

**Saving the result**

In [27]:
reviews.to_csv('data/ReviewNew.csv', index_label='id')
products.to_csv('data/ProductNew.zip', index=False)